In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras.layers import SpatialDropout2D,Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate,AveragePooling2D, UpSampling2D, BatchNormalization, Activation, add,Dropout,Permute,ZeroPadding2D,Add, Reshape
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import ELU, LeakyReLU, ReLU, PReLU
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import applications, optimizers, callbacks
import matplotlib
import tensorflow as tf
from tensorflow.keras.layers import *
from model import DCUNet

In [3]:
X = []
Y = []

for i in range(612):
    
    path = 'C:\\Users\\Usuario\\Desktop\\\DCUnet_tf_2_4\\CVC-ClinicDB\\Original\\'+ str(i+1)+'.tif'
    img = cv2.imread(path,1)
    resized_img = cv2.resize(img,(128, 96), interpolation = cv2.INTER_CUBIC)
    
    X.append(resized_img)
    
for i in range(612):
    
    path2 = 'C:\\Users\\Usuario\\Desktop\\\DCUnet_tf_2_4\\CVC-ClinicDB\\Ground Truth\\' + str(i+1)+'.tif'
    msk = cv2.imread(path2,0)

    resized_msk = cv2.resize(msk,(128, 96), interpolation = cv2.INTER_CUBIC)
    
    Y.append(resized_msk)
    
X = np.array(X)
Y = np.array(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
Y_train = Y_train.reshape((Y_train.shape[0],Y_train.shape[1],Y_train.shape[2],1))
Y_test = Y_test.reshape((Y_test.shape[0],Y_test.shape[1],Y_test.shape[2],1))


X_train = X_train / 255
X_test = X_test / 255
Y_train = Y_train / 255
Y_test = Y_test / 255

Y_train = np.round(Y_train,0)	
Y_test = np.round(Y_test,0)	

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(489, 96, 128, 3)
(489, 96, 128, 1)
(123, 96, 128, 3)
(123, 96, 128, 1)


In [4]:
# different loss functions
def dice_coef(y_true, y_pred):
    smooth = 1.0  #0.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jacard(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum ( y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

    return intersection/union

def dice_coef_loss(y_true,y_pred):
    return 1 - dice_coef(y_true,y_pred)

def iou_loss(y_true,y_pred):
    return 1 - jacard(y_true, y_pred)

def tversky(y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1-y_pred_pos))
    false_pos = K.sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.75
    smooth = 1
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)
def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true,y_pred)
def focal_tversky(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return K.pow((1-pt_1), gamma)

# training

def saveModel(model):

    model_json = model.to_json()

    try:
        os.makedirs('models')
    except:
        pass
    
    fp = open('models/modelP.json','w')
    fp.write(model_json)
    model.save('models/modelW.h5')


def evaluateModel(model, X_test, Y_test, batchSize):
    
    try:
        os.makedirs('results')
    except:
        pass 
    
    yp = model.predict(x=X_test, batch_size=batchSize, verbose=1)
    yp = np.round(yp,0)
    
    for i in range(10):

        plt.figure(figsize=(20,10))
        plt.subplot(1,3,1)
        plt.imshow(X_test[i])
        plt.title('Input')
        plt.subplot(1,3,2)
        plt.imshow(Y_test[i].reshape(Y_test[i].shape[0],Y_test[i].shape[1]))
        plt.title('Ground Truth')
        plt.subplot(1,3,3)
        plt.imshow(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        plt.title('Prediction')

        intersection = yp[i].ravel() * Y_test[i].ravel()
        union = yp[i].ravel() + Y_test[i].ravel() - intersection

        jacard = (np.sum(intersection)/np.sum(union))  
        plt.suptitle('Jacard Index'+ str(np.sum(intersection)) +'/'+ str(np.sum(union)) +'='+str(jacard))

        plt.savefig('results/'+str(i)+'.png',format='png')
        plt.close()
    
    jacard = 0
    dice = 0
    
    
    for i in range(len(Y_test)):
        yp_2 = yp[i].ravel()
        y2 = Y_test[i].ravel()
        
        intersection = yp_2 * y2
        union = yp_2 + y2 - intersection

        jacard += (np.sum(intersection)/np.sum(union))  

        dice += (2. * np.sum(intersection) ) / (np.sum(yp_2) + np.sum(y2))

    
    jacard /= len(Y_test)
    dice /= len(Y_test)
    


    print('Jacard Index : '+str(jacard))
    print('Dice Coefficient : '+str(dice))
    

    fp = open('models/log.txt','a')
    fp.write(str(jacard)+'\n')
    fp.close()

    fp = open('models/best.txt','r')
    best = fp.read()
    fp.close()

    if(jacard>float(best)):
        print('***********************************************')
        print('Jacard Index improved from '+str(best)+' to '+str(jacard))
        print('***********************************************')
        fp = open('models/best.txt','w')
        fp.write(str(jacard))
        fp.close()

        saveModel(model)

In [5]:
# def trainStep(model, X_train, Y_train, X_test, Y_test, epochs, batchSize):

    
#     for epoch in range(epochs):
#         print('Epoch : {}'.format(epoch+1))
#         model.fit(x=X_train, y=Y_train, batch_size=batchSize, epochs=1, verbose=1)     

#         evaluateModel(model,X_test, Y_test,batchSize)

#     return model 

In [6]:
# model = DCUNet(height=96, width=128, channels=3)

# model.compile(optimizer='adam', loss=focal_tversky, metrics=[dice_coef, jacard, 'accuracy'])
# #binary_crossentropy
# model.summary()
# saveModel(model)

In [7]:
# fp = open('models/log.txt','w')
# fp.close()
# fp = open('models/best.txt','w')
# fp.write('-1.0')
# fp.close()
    
# trainStep(model, X_train, Y_train, X_test, Y_test, epochs=150, batchSize=4)

In [8]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [9]:
model = DCUNet(height=96, width=128, channels=3)

model.compile(optimizer='adam', loss=focal_tversky, metrics=[dice_coef, jacard, 'accuracy'])
#binary_crossentropy
model.summary()
saveModel(model)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 128, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 96, 128, 8)   216         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 96, 128, 8)   216         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 96, 128, 8)   24          conv2d[0][0]                     
______________________________________________________________________________________________

In [10]:
history = model.fit(x=X_train, y=Y_train,
                     epochs=150, batch_size=1, callbacks=[earlyStop],
                     verbose=1)

Epoch 1/150
489/489 [==============================] - 61s 100ms/step - loss: 0.7485 - dice_coef: 0.2187 - jacard: 0.1302 - accuracy: 0.7094
Epoch 2/150
489/489 [==============================] - 49s 101ms/step - loss: 0.7471 - dice_coef: 0.2227 - jacard: 0.1330 - accuracy: 0.7239
Epoch 3/150
489/489 [==============================] - 47s 96ms/step - loss: 0.7342 - dice_coef: 0.2372 - jacard: 0.1431 - accuracy: 0.7913
Epoch 4/150
489/489 [==============================] - 48s 98ms/step - loss: 0.7211 - dice_coef: 0.2544 - jacard: 0.1545 - accuracy: 0.8265
Epoch 5/150
489/489 [==============================] - 46s 95ms/step - loss: 0.7155 - dice_coef: 0.2604 - jacard: 0.1580 - accuracy: 0.8563
Epoch 6/150
489/489 [==============================] - 47s 95ms/step - loss: 0.7085 - dice_coef: 0.2733 - jacard: 0.1665 - accuracy: 0.8707
Epoch 7/150
489/489 [==============================] - 53s 107ms/step - loss: 0.6973 - dice_coef: 0.2869 - jacard: 0.1764 - accuracy: 0.8939
Epoch 8/150
489/4